# BM25 >> MonoT5 Pipeline

### Step 1: Import everything and load variables

In [2]:
import pyterrier as pt
import pandas as pd
from tira.third_party_integrations import ensure_pyterrier_is_loaded, get_input_directory_and_output_directory, persist_and_normalize_run
import json
from tqdm import tqdm

ensure_pyterrier_is_loaded()
input_directory, output_directory = get_input_directory_and_output_directory('./sample-input-full-rank')


I will use a small hardcoded example located in ./sample-input-full-rank.
The output directory is /tmp/


### Step 2: Load the Data

In [3]:
print('Step 2: Load the data.')

queries = pt.io.read_topics(input_directory + '/queries.xml', format='trecxml')

documents = (json.loads(i) for i in open(input_directory + '/documents.jsonl', 'r'))


Step 2: Load the data.


### Step 3: Create the Index

In [4]:
print('Step 3: Create the Index.')

!rm -Rf ./index
iter_indexer = pt.IterDictIndexer("./index", meta={'docno' : 100, 'text': 10240})
index_ref = iter_indexer.index(tqdm(documents))

Step 3: Create the Index.


5it [00:00, 48.24it/s]


### Step 4: Create Run

In [7]:
from pyterrier_t5 import MonoT5ReRanker
import os

bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25", metadata=['docno', 'text'])

mono_t5 = MonoT5ReRanker(model=os.environ['MODEL_NAME'], tok_model=os.environ['TOKENIZER_NAME'])

pipeline = bm25 % 1000 >> mono_t5

In [8]:
run = pipeline(queries)

monoT5: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.26batches/s]


### Step 5: Persist Run

In [9]:
print('Step 5: Persist Run.')

persist_and_normalize_run(run, output_file=output_directory, system_name='MonoT5', depth=1000)

Step 5: Persist Run.
